<a href="https://colab.research.google.com/github/BernieT7/ML/blob/main/TransLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests
import zipfile
import torch
from torch.utils.data import Dataset
from pathlib import Path
from PIL import Image

In [2]:
url = "https://firebasestorage.googleapis.com/v0/b/grandmacan-2dae4.appspot.com/o/ML_data%2Fone_piece_full.zip?alt=media&token=937656fd-f5c1-44f5-b174-1e2d590b8ef3"

with open("one_piece_full.zip", "wb") as f:
  req = requests.get(url)
  f.write(req.content)

with zipfile.ZipFile("one_piece_full.zip", "r") as zip_file:
  zip_file.extractall("one_piece_full")

In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [4]:
def accuracy_fn(y_pred, y_true):
  correct_num = (y_pred==y_true).sum()
  acc = correct_num / len(y_true) * 100
  return acc

def train_step(dataloader, model, cost_fn, optimizer, accuracy_fn, device):
  train_cost = 0
  train_acc = 0
  for batch, (x, y) in enumerate(dataloader):
    x = x.to(device)
    y = y.to(device)
    model.to(device)

    model.train()

    y_pred = model(x)

    cost = cost_fn(y_pred, y)

    train_cost += cost
    train_acc += accuracy_fn(y_pred.argmax(dim=1), y)

    optimizer.zero_grad()

    cost.backward()

    optimizer.step()

  train_cost /= len(train_dataloader)
  train_acc /= len(train_dataloader)

  print(f"\nTrain Cost: {train_cost:.4f}, Train Acc: {train_acc:.2f}")


def test_step(dataloader, model, cost_fn, accuracy_fn, device):
  test_cost = 0
  test_acc = 0
  model.eval()
  with torch.inference_mode():
    for x, y in dataloader:
      x = x.to(device)
      y = y.to(device)
      model.to(device)

      test_pred = model(x)

      test_cost += cost_fn(test_pred, y)
      test_acc += accuracy_fn(test_pred.argmax(dim=1), y)

    test_cost /= len(test_dataloader)
    test_acc /= len(test_dataloader)

  print(f"Test Cost: {test_cost:.4f}, Test Acc: {test_acc:.2f} \n")

In [5]:
class ImageDataset(Dataset):
  def __init__(self, root, train, transform=None):

    if train:
      image_root = Path(root) / "train"
    else:
      image_root = Path(root) / "test"

    with open(Path(root) / "classnames.txt", "r") as f:
      lines = f.readlines()
      self.classes = [line.strip() for line in lines]

    self.paths = [i for i in image_root.rglob("*") if i.is_file()]
    self.transform = transform

  def __getitem__(self, index):
    img = Image.open(self.paths[index]).convert("RGB")
    class_name = self.paths[index].parent.name
    class_idx = self.classes.index(class_name)

    if self.transform:
      return self.transform(img), class_idx
    else:
      return img, class_idx


  def __len__(self):
    return len(self.paths)

In [6]:
import torchvision

weights = torchvision.models.EfficientNet_B1_Weights.DEFAULT              # 取得別人訓練的模型的參數
model = torchvision.models.efficientnet_b1(weights=weights)               # 使用EfficientNet_B1模型並套用訓練好的參數
model

Downloading: "https://download.pytorch.org/models/efficientnet_b1-c27df63c.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_b1-c27df63c.pth
100%|██████████| 30.1M/30.1M [00:00<00:00, 121MB/s]


EfficientNet(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): Conv2dNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): Conv2dNormActivat

In [7]:
efficientnet_b1_transforms = weights.transforms()                      # 查看此模型是如何轉換圖片的

In [8]:
train_dataset = ImageDataset(root="one_piece_full",
              train=True,
              transform=efficientnet_b1_transforms               # 此模型怎模轉的，我們也要將我們手中的資料做同樣的轉換
)

test_dataset = ImageDataset(root="one_piece_full",
              train=False,
              transform=efficientnet_b1_transforms
)

In [9]:
from torch.utils.data import DataLoader

BATCH_SIZE = 16

train_dataloader = DataLoader(dataset=train_dataset,
                batch_size=BATCH_SIZE,
                shuffle=True
)

test_dataloader = DataLoader(dataset=test_dataset,
                batch_size=BATCH_SIZE,
                shuffle=False
)

In [10]:
len(train_dataloader), len(test_dataloader)

(189, 47)

In [11]:
!pip install torchinfo
from torchinfo import summary                             # 此套件可以讓我們更方便觀看複雜的類神經網路結構

In [12]:
summary(model=model,
    input_size=(16, 3, 64, 64),
    col_names=["input_size", "output_size", "num_params", "trainable"],
    row_settings=["var_names"]                            # 可以顯示出各層函數名稱，方便後面我們改變某些層的input, output
)

Layer (type (var_name))                                      Input Shape               Output Shape              Param #                   Trainable
EfficientNet (EfficientNet)                                  [16, 3, 64, 64]           [16, 1000]                --                        True
├─Sequential (features)                                      [16, 3, 64, 64]           [16, 1280, 2, 2]          --                        True
│    └─Conv2dNormActivation (0)                              [16, 3, 64, 64]           [16, 32, 32, 32]          --                        True
│    │    └─Conv2d (0)                                       [16, 3, 64, 64]           [16, 32, 32, 32]          864                       True
│    │    └─BatchNorm2d (1)                                  [16, 32, 32, 32]          [16, 32, 32, 32]          64                        True
│    │    └─SiLU (2)                                         [16, 32, 32, 32]          [16, 32, 32, 32]          --                

In [17]:
from torch import nn
model.classifier[1] = nn.Linear(in_features=1280, out_features=18, bias=True).to(device)     # 改變最後一層輸出量

In [18]:
# 改變了最後一層後，最後一層的參數要重新訓練，馾其他層不用跟著訓練，所以要鎖住
# 鎖住的方法就是把前面的所有層的所有參數(features)的追蹤梯度取消！
for params in model.features.parameters():                       #這樣就鎖住了
  params.requires_grad = False

In [19]:
# 設定訓練最後一層的cost function, optimizer
cost_fn = nn.CrossEntropyLoss()                             # cost function
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)            # optimizer

In [20]:
# 開始訓練
from tqdm.auto import tqdm

epochs = 10

for epoch in tqdm(range(epochs)):
  print(f"Epoch: {epoch}\n-------")

  train_step(train_dataloader, model, cost_fn, optimizer, accuracy_fn, device)

  test_step(test_dataloader, model, cost_fn, accuracy_fn, device)


  0%|          | 0/10 [00:00<?, ?it/s]

Epoch: 0
-------


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(



Train Cost: 2.3328, Train Acc: 52.55
Test Cost: 1.9274, Test Acc: 67.78 

Epoch: 1
-------

Train Cost: 1.5822, Train Acc: 71.68
Test Cost: 1.4895, Test Acc: 72.97 

Epoch: 2
-------

Train Cost: 1.2427, Train Acc: 76.80
Test Cost: 1.2825, Test Acc: 74.03 

Epoch: 3
-------

Train Cost: 1.0197, Train Acc: 80.14
Test Cost: 1.0895, Test Acc: 77.62 

Epoch: 4
-------

Train Cost: 0.8945, Train Acc: 81.32
Test Cost: 0.9914, Test Acc: 77.75 

Epoch: 5
-------

Train Cost: 0.8032, Train Acc: 83.62
Test Cost: 0.9230, Test Acc: 79.08 

Epoch: 6
-------

Train Cost: 0.7391, Train Acc: 84.04
Test Cost: 0.8800, Test Acc: 79.48 

Epoch: 7
-------

Train Cost: 0.6741, Train Acc: 85.20
Test Cost: 0.8278, Test Acc: 81.48 

Epoch: 8
-------

Train Cost: 0.6232, Train Acc: 85.52
Test Cost: 0.8158, Test Acc: 80.28 

Epoch: 9
-------

Train Cost: 0.5864, Train Acc: 86.92
Test Cost: 0.7719, Test Acc: 80.68 



In [28]:
# 實際應用的圖片也要照著當初訓練模型時處理圖片的方式去處理(格式、 大小等等...)
img = Image.open("usopp_test.jpg").convert("RGB")
img = efficientnet_b1_transforms(img)
img.shape
img = img.reshape(-1, 3, 240, 240)            # 因為這張圖片的shape少了batch的維度所以要reshape
model.eval()
with torch.inference_mode():
  y_pred = model(img.to(device))
y_pred = torch.softmax(y_pred, dim=1)
class_idx = y_pred.argmax(dim=1)
train_dataset.classes[class_idx]

'Usopp'